In [ ]:
!pip -q install wandb
!pip -q install torch-summary
!wget -q https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q nature_12K.zip
!rm nature_12K.zip
!mv ./inaturalist_12K/val ./inaturalist_12K/test

In [ ]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"
os.environ['WANDB_CONSOLE'] = 'off'
wandb.login()

In [ ]:
import torch
import numpy as np
import random

from hyperparams import bayes_config_instance1, bayes_config_instance2, default_config_instance1, default_config_instance2, part_A_best_config, default_best_config_part_a
from agent import Agent
from cnn import CNN
from utils import Utils
from guided_backprop import Guided_backprop
from torch-summary import summary

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED = 123
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

## Find hyperparams

In [ ]:
train_path = "/content/inaturalist_12K/train"
test_path = "/content/inaturalist_12K/test"
BATCH_SIZE = 256
EPOCHS = 10
IMG_DIM = 128

In [ ]:
def single_sweep(default_config):
    val_acc = 0
    wandb.init(config=default_config)
    config = wandb.config

    HYPERPARAMS = config._as_dict()
    wandb.run.name = None
    wandb.run.name = HYPERPARAMS["activation"] + "_nfil_" + str(HYPERPARAMS['n_filters']) + "_filorg_" + str(HYPERPARAMS['conv_filter_org']) + "_BN_" + str(HYPERPARAMS['batch_norm']) + "_filsize_" + str(HYPERPARAMS['conv_filter_size']) + "_dout_" + str(HYPERPARAMS['dropout']) + "_data_aug_" + str(HYPERPARAMS['data_aug']) + "_mlp_neurons_" + str(HYPERPARAMS['n_mlp_neurons'])

    utils_agent = Utils(train_path, test_path, BATCH_SIZE, (IMG_DIM, IMG_DIM), train_transform=HYPERPARAMS['data_aug'])
    model = CNN((1, 3, IMG_DIM, IMG_DIM), HYPERPARAMS)
    agent = Agent(model, utils_agent, EPOCHS)
    val_acc = agent.train() # your model training code here
    wandb.log({'val_acc': val_acc.cpu().item()})
    wandb.finish(quiet=True)

In [ ]:
# run the instance_1 sweep
sweep_id = wandb.sweep(bayes_config_instance1, project="test", entity="kbdl")
wandb.agent(sweep_id, function=single_sweep(default_config_instance1), count=15)

In [ ]:
# run the instance_2 sweep
sweep_id = wandb.sweep(bayes_config_instance2, project="test", entity="kbdl")
wandb.agent(sweep_id, function=single_sweep(default_config_instance2), count=8)

## Best hyperparams

In [ ]:
def best_sweep():
    val_acc, test_acc = 0, 0
    wandb.init(config=default_best_config_part_a)
    config = wandb.config

    HYPERPARAMS = config._as_dict()
    wandb.run.name = None
    wandb.run.name = "GBP" + HYPERPARAMS["activation"] + "_nfil_" + str(HYPERPARAMS['n_filters']) + "_filorg_" + str(HYPERPARAMS['conv_filter_org']) + "_BN_" + str(HYPERPARAMS['batch_norm']) + "_filsize_" + str(HYPERPARAMS['conv_filter_size']) + "_dout_" + str(HYPERPARAMS['dropout']) + "_data_aug_" + str(HYPERPARAMS['data_aug']) + "_mlp_neurons_" + str(HYPERPARAMS['n_mlp_neurons'])

    utils_agent = Utils(train_path, test_path, BATCH_SIZE, (IMG_DIM, IMG_DIM), train_transform=HYPERPARAMS['data_aug'])
    model = CNN((1, 3, IMG_DIM, IMG_DIM), HYPERPARAMS)
    #summary(model, (3, IMG_DIM, IMG_DIM))
    agent = Agent(model, utils_agent, EPOCHS)

    val_acc = agent.train() # your model training code here
    _, test_acc = agent.evaluate(test_data=True)
    wandb.log({'val_acc': val_acc.cpu().item(), 'test_accuracy': test_acc.cpu().item()})

    utils_agent.visualize_filters_cnn(agent.model)
    guided_bp = Guided_backprop(agent.model, utils_agent)
    guided_bp.visualize(utils_agent.get_rand_image(dataloader_type='test'))

    wandb.finish(quiet=True)

In [ ]:
# run the best sweep
sweep_id = wandb.sweep(part_A_best_config, project="test", entity="kbdl")
wandb.agent(sweep_id, function=best_sweep, count=1)